In [1]:
import pyexasol
import pandas as pd
import sys

### Exasol Connection

In [2]:
ip = "localhost:8563"
C = pyexasol.connect(dsn=ip, user='sys', password='exasol', protocol_version=pyexasol.PROTOCOL_V1)

### All Tables in DB

In [3]:
tables = C.export_to_pandas("SELECT * FROM EXA_ALL_TABLES")
tables

,TABLE_SCHEMA,TABLE_NAME,TABLE_OWNER,TABLE_OBJECT_ID,TABLE_IS_VIRTUAL,TABLE_HAS_DISTRIBUTION_KEY,TABLE_ROW_COUNT,DELETE_PERCENTAGE,TABLE_COMMENT
0,AOL_SCHEMA,ERR_TAB,SYS,156742656,0,0,0,0,NaN
1,AOL_SCHEMA,QUERYDIM,SYS,156759040,0,0,10154742,0,NaN
2,AOL_SCHEMA,ANONIDDIM,SYS,157811712,0,0,657427,0,NaN
3,AOL_SCHEMA,URLDIM,SYS,157826048,0,0,4051750,0,NaN
4,AOL_SCHEMA,FACTS,SYS,157850624,0,0,36389566,0,NaN
5,AOL_SCHEMA,URL_CATEGORY,SYS,159348736,0,0,3965429,0,NaN
6,AOL_SCHEMA,TIMEDIM,SYS,159379456,0,0,6927038,0,NaN
7,AOL_SCHEMA,DMOZ_CATEGORIES,SYS,159407104,0,0,779122,0,NaN
8,AOL_SCHEMA,CITY,SYS,199041024,0,0,50,0,NaN
9,AOL_SCHEMA,ZIP_CODE,SYS,199074816,0,0,10600,0,NaN


### Drop unneeded tables

In [26]:
C.execute("DROP TABLE AOL_SCHEMA.ACTOR")
C.execute("DROP TABLE AOL_SCHEMA.MOVIE")
C.execute("DROP TABLE AOL_SCHEMA.IS_WINNER")
C.execute("DROP TABLE AOL_SCHEMA.OSCAR_CATEGORY")
C.execute("DROP TABLE AOL_SCHEMA.OSCAR_MOVIES")
C.execute("DROP TABLE AOL_SCHEMA.OSCAR_NOMINEE")
C.execute("DROP TABLE AOL_SCHEMA.PLAYS_IN")

<ExaStatement session_id=1748381385000263339 stmt_idx=9>

In [14]:
#C.execute("DROP TABLE AOL_SCHEMA.DRUG_ALIAS")

### Drugs and Slangwords

In [5]:
# Source: https://nida.nih.gov/research-topics/commonly-used-drugs-charts
# Only illegal/prescription and specific drugs
drugs = ["Ayahuasca", "Cannabis", "Benzodiazepine", "Cocaine", "GHB", "Heroin", "Ketamine", "Khat", "Kratom", "LSD", "MDMA", "Mescaline",
         "Methamphetamine", "DXM", "PCP", "Prescription Opioids", "Prescription Stimulants", "Psilocybin",
         "Salvia", "Anabolic Steroids", "Synthetic Cannabinoids", "Synthetic Cathinones"]
aliases = {drug: [drug] for drug in drugs}

In [6]:
categorys = ["Hallucinogens", "Cannabis", "CNS Depressants", "CNS Stimulants", "CNS Depressants", "Narcotic Analgesics", "Dissociative Anesthetics",
             "CNS Stimulants", "Dissociative Anesthetics", "Hallucinogens", "CNS Stimulants", "Hallucinogens", "CNS Stimulants", "Dissociative Anesthetics",
             "Dissociative Anesthetics", "Narcotic Analgesics", "CNS Stimulants", "Hallucinogens", "Hallucinogens", "Unknown", "Cannabis", "CNS Stimulants"]

In [15]:
# most slang words from urbansthesaurus or https://adf.org.au/drug-facts
# I tried to skip indistinct words to avoid false positives
aliases["Ayahuasca"] += ["ayawaska", "changa", "deemster", "dmt"]
aliases["Cannabis"] += ["hippie cabbage", "ganja", "weed", "thc"]
aliases["Benzodiazepine"] += ["Benzo", "Benzos", "surrey", "xanax", "Rohypnol"]
aliases["Cocaine"] += ["nosebag", "kitchen snow"]
aliases["GHB"] += ["gamma hydroxybutyrate", "gbl", "gbh", "liquid ecstasy", "liquid x",
                 "liquid e"]
aliases["Heroin"] += ["chiva", "brownstone", "scag"]
aliases["Ketamine"] += ["special k", "kitty flip", "vitamin k", "k-bomb"]
aliases["Khat"] += ["Qat", "kat", "qaad"]
aliases["Kratom"] += ["kratz"]
#aliases["LSD"] += ["Acid"]
aliases["MDMA"] += ["Ecstasy", "Eckies", "XTC", "pingers", "bikkies", "flippers", "molly"]
aliases["Mescaline"] += ["mesc", "microdot", "synthmesc", "peyote", "cactus flip"]
aliases["Methamphetamine"] += ["Meth", "Crystal Meth", "shabu"]
aliases["DXM"] += ["robotrip", "robo-trip", "roboshits", "robotripping"]
aliases["PCP"] += ["angel dust", "ashy larry", "angeldust"]
aliases["Prescription Opioids"] += ["Oxy", "Percs", "Codeine", "Fentanyl", "Vicodin", "Hydromorphone", 
                                  "Meperidine", "Methadone", "Oxycodone", "Oxymorphone"]
aliases["Prescription Stimulants"] += ["Amphetamine", "Aderall", "Methylphenidate", "Concerta", "Ritalin"]
aliases["Psilocybin"] += ["Shrooms", "Magic mushrooms"]
#aliases["Rohypnol"] += ["roofies", "roofie"]
aliases["Salvia"] += ["Salvia divinorum"]
aliases["Anabolic Steroids"] += ["Anabolic", "Anabolica", "Nandrolone", "oxandrolone", "oxymetholone",
                      "testosterone cypionate", "roids"]
aliases["Synthetic Cannabinoids"] += ["K2"]
aliases["Synthetic Cathinones"] += ["Bath Salts", "dones", "Mephedrone", "Methylone"]

### Create table for Drugs

In [10]:
C.execute(
    """
    CREATE TABLE AOL_SCHEMA.DRUG(
        ID int IDENTITY PRIMARY KEY,
        NAME VARCHAR(100),
        CATEGORY VARCHAR(100)
    );
    """
)

<ExaStatement session_id=1749114811869395740 stmt_idx=4>

### Create Table for Drug Aliases

In [16]:
C.execute(
    """
    CREATE TABLE AOL_SCHEMA.DRUG_ALIAS(
        ID int CONSTRAINT DRUG_ID REFERENCES AOL_SCHEMA.DRUG (ID) DISABLE,
        ALIAS VARCHAR(100)
    );
    """
)

<ExaStatement session_id=1749148092605066629 stmt_idx=115>

### Insert into Drugs

In [12]:
for i, drug in enumerate(drugs):
    insert = """
                INSERT INTO AOL_SCHEMA.DRUG(NAME, CATEGORY) VALUES
                ('{}', '{}');
            """.format(drug, categorys[i])
    C.execute(insert)

In [17]:
drugs_df = C.export_to_pandas("SELECT * FROM AOL_SCHEMA.DRUG")
drugs_df

,ID,NAME,CATEGORY
0,1,Ayahuasca,Hallucinogens
1,2,Cannabis,Cannabis
2,3,Benzodiazepine,CNS Depressants
3,4,Cocaine,CNS Stimulants
4,5,GHB,CNS Depressants
5,6,Heroin,Narcotic Analgesics
6,7,Ketamine,Dissociative Anesthetics
7,8,Khat,CNS Stimulants
8,9,Kratom,Dissociative Anesthetics
9,10,LSD,Hallucinogens


### Insert Drug Aliases

In [18]:
for i, row in drugs_df.iterrows():
    drug = row['NAME']
    ID = row['ID']
    
    drug_aliases = aliases[drug]
    for alias in drug_aliases:
        insert = """
                INSERT INTO AOL_SCHEMA.DRUG_ALIAS(ID, ALIAS) VALUES
                ('{}', '{}');
                """.format(ID, alias)
        C.execute(insert)

### Test join

In [12]:
drugs_with_aliases = C.export_to_pandas("""
    SELECT * FROM AOL_SCHEMA.DRUG D
    INNER JOIN AOL_SCHEMA.DRUG_ALIAS DA
    ON
        D.ID = DA.ID
"""
)
drugs_with_aliases

,ID,NAME,CATEGORY,ID.1,ALIAS
0,1,Ayahuasca,Hallucinogens,1,Ayahuasca
1,1,Ayahuasca,Hallucinogens,1,ayawaska
2,1,Ayahuasca,Hallucinogens,1,changa
3,1,Ayahuasca,Hallucinogens,1,deemster
4,1,Ayahuasca,Hallucinogens,1,dmt
...,...,...,...,...,...
103,22,Synthetic Cathinones,CNS Stimulants,22,Synthetic Cathinones
104,22,Synthetic Cathinones,CNS Stimulants,22,Bath Salts
105,22,Synthetic Cathinones,CNS Stimulants,22,dones
106,22,Synthetic Cathinones,CNS Stimulants,22,Mephedrone
